In [3]:
import yfinance as yf 
import pandas as pd
import numpy as np
from datetime import datetime

import quandl

import sys
import urllib

In [24]:
def import_MSCI(name):
    '''Function to clean data from MSCI
    https://www.msci.com/end-of-day-data-country
    '''
    
    
    path = "Data/" + name + ".xls"
    imp_tmp = pd.read_excel(path)
    price_type = imp_tmp.loc[1][1]
    
    imp = imp_tmp[6:]
    imp.columns = ['Date', name]
    #imp['Date'] = pd.to_datetime(imp['Date'], errors='coerce')
    imp.loc[0] = pd.to_datetime(imp['Date'], errors='coerce')
    imp = imp.dropna()
    
    imp = imp.set_index('Date')
    #imp.index = imp.index.strftime('%Y/%m')
    
    print(price_type)
    
    return(imp)

# Data Collection

## Equity

### Monthly

In [87]:
USA = import_MSCI("USA")
Pacific_ex_Japan = import_MSCI("PACIFIC_ex_JAPAN")
Japan = import_MSCI("JAPAN")
Sweden = import_MSCI("SWEDEN")
Emerging_Markets = import_MSCI("EMERGING_MARKETS")
RUSSIA = import_MSCI("RUSSIA")

Gross
Gross
Gross
Gross
Gross
Gross


In [88]:
Emerging_Markets


,EMERGING_MARKETS
Date,
1987-12-31,100
1988-01-29,109.838
1988-02-29,110.211
1988-03-31,122.046
1988-04-29,128.561
...,...
2020-02-28,2322.26
2020-03-31,1965.16
2020-04-30,2145.5


### Daily

In [21]:
SP500_daily_tmp = yf.download("^SP500TR")
SP500_daily= pd.DataFrame(SP500_daily_tmp['Close'])
SP500_daily.columns = ['SP500']

[*********************100%***********************]  1 of 1 completed


## Bonds

### Yearly

In [27]:
# Only says T-bond, 10 yr?, 30 yr?
d = pd.read_html('http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html')
T_tmp = d[0]
T_tmp = pd.DataFrame(T_tmp[7][:])

years =  pd.Series(range(1926,2020))
T_tmp.insert(0, "Year", years, True)
T_tmp.insert(2, "Month", 1, True)
T_tmp.insert(3, "Day", 1, True)
T_tmp = T_tmp[2:]
T_tmp.columns = ['Year','T bond', 'Month', 'Day'] 
T_tmp['T bond'] = T_tmp['T bond'].str.replace('$', '')
T_tmp['T bond'] = T_tmp['T bond'].str.replace(',', '')
T_tmp['T bond'] = pd.to_numeric(T_tmp['T bond'])

T_tmp['Year'] = pd.to_datetime(T_tmp['Year']*10000+T_tmp['Month']*100+T_tmp['Day'],format='%Y%m%d')
T_tmp = T_tmp.set_index('Year')
T_bond_yearly = pd.DataFrame(T_tmp['T bond'])
T_bond_yearly

,T bond
Year,
1928-01-01,100.84
1929-01-01,105.07
1930-01-01,109.85
1931-01-01,107.03
1932-01-01,116.44
...,...
2015-01-01,7061.89
2016-01-01,7110.65
2017-01-01,7309.87


## Alternatives

In [8]:
#Gold
Gold_tmp = quandl.get("LBMA/GOLD", authtoken="6rNSyTM5R4g7_B7XSyQW")
Gold = pd.DataFrame(Gold_tmp["USD (AM)"])
Gold

,USD (AM)
Date,
1968-01-02,35.18
1968-01-03,35.16
1968-01-04,35.14
1968-01-05,35.14
1968-01-08,35.14
...,...
2020-07-16,1804.60
2020-07-17,1802.90
2020-07-20,1810.30


In [ ]:
#Long Vol(Monthly)
long_vol_imp = pd.read_excel (r'https://www.eurekahedge.com/Indices/ExportIndexReturnsToExcel?IndexType=Eurekahedge&IndexId=640')

long_vol = long_vol_imp[3:]
long_vol.index= pd.to_datetime(long_vol_imp[3:]['Unnamed: 0'])
long_vol = pd.DataFrame(long_vol['Unnamed: 2'])
long_vol.columns = ["Long Vol"]